# TVM 执行器

In [1]:
from tvm.relay.backend import Executor

def test_create_executor():
    executor = Executor("aot")
    assert executor.name == "aot"


def test_create_executor_with_options():
    executor = Executor("aot", {"interface-api": "c"})
    assert executor.name == "aot"
    assert executor["interface-api"] == "c"


def test_create_executor_with_default():
    executor = Executor("graph")
    assert not executor["link-params"]


def test_attr_check():
    executor = Executor("aot", {"interface-api": "c"})
    assert "woof" not in executor
    assert "interface-api" in executor

In [2]:
from tvm.relay import var, const, create_executor
from tvm.relay.op import debug


_test_debug_hit = False


def test_debug():
    global _test_debug_hit
    x = var("x", shape=(), dtype="int32")
    _test_debug_hit = False

    def did_exec(x):
        global _test_debug_hit
        _test_debug_hit = True

    prog = debug(x, debug_func=did_exec)
    result = create_executor().evaluate(prog, {x: const(1, "int32")})
    assert _test_debug_hit
    assert result.numpy() == 1

def test_debug_with_expr():
    global _test_debug_hit
    _test_debug_hit = False
    x = var("x", shape=(), dtype="int32")
    _test_debug_hit = False

    def did_exec(x):
        global _test_debug_hit
        _test_debug_hit = True

    prog = debug(x + x * x, debug_func=did_exec)
    result = create_executor().evaluate(prog, {x: const(2, "int32")})
    assert _test_debug_hit
    assert result.numpy() == 6

In [ ]:
test_debug()
test_debug_with_expr()